In [ ]:
import csv
from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions
import pandas as pd
import csv
import json
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import pandas as pd
import time

In [ ]:
test = []
names = []
flipPrices = []
prodNames = []
info,price = [],[]
url = "https://flipkart.com/search?q="
# query to search for.
q = input("Enter product name (Searching on flipkart)")
q = q.replace(" ","+")
#     q = input("Enter a query: ")
file_name = q.replace(" ", "_")
# response recieved in bytes
resp = requests.get(url+q)
# parsing response content using BeautifulSoup class, so that we can perform operations on it.
parsed_html = bs(resp.content, 'html.parser')
# data cleaning
raw_data = parsed_html.find("script", attrs={"id":"is_script"})
data = raw_data.contents[0].replace("window.__INITIAL_STATE__ = ","").replace(";","")
json_data = json.loads(data)
req_data = json_data["pageDataV4"]["page"]["data"]["10003"]   #[10]["widget"]["data"]["products"][3]["productInfo"]
#req_json_data = json_data["seoMeta"]["answerBox"]["data"]["renderableComponents"][0]["value"]["data"]

In [ ]:
data_list = []
# print(len(req_data))
try:
    for i in range(1, len(req_data)):
        d = {}
        jd = req_data[i]["widget"]["data"]["products"]
        # print(len(jd))
        # print("i: ", i, end="\n")
        for j in range(len(jd)):
            jd2 = jd[j]["productInfo"]["value"]

            d["title"] = jd2["titles"]["title"]
            d["keySpecs"] = jd2["keySpecs"]
            d["rating"] = jd2["rating"]["average"]
            d["ratingCount"] = jd2["rating"]["count"]
            d["price"] = jd2["pricing"]["finalPrice"]["value"]
            #d["warranty"] = jd2["warrantySummary"]
            d["url"] = jd2["smartUrl"]

            # You can uncomment below lines if you want to print json output on terminal

            # print("Title: ",jd2["titles"]["title"],end="\n")
            # print("key specs: ", jd2["keySpecs"], end="\n")
            # print("Rating: ", jd2["rating"]["average"], end="\n")
            # print("Total ratings: ", jd2["rating"]["count"], end="\n")
            # print("Price: ", jd2["pricing"]["finalPrice"]["value"],end="\n")
            # print("warranty: ", jd2["warrantySummary"], end="\n")
            # print("Smart url: ", jd2["smartUrl"], end="\n")
        data_list.append(d)
        
except:
    pass 
# dumping data to result.json file
    # print(list(data_list))
with open("flipkart"+'.json', 'w') as fp:
    json.dump(data_list, fp)

# Now let us write our data to csv file
data_file = open("flipkart"+'.csv', 'w') 

# create the csv writer object 
csv_writer = csv.writer(data_file) 

# Counter variable used for writing  
# headers to the CSV file 
count = 0




In [ ]:
for data in data_list:
    if count == 0: 

        # Writing headers of CSV file 
        header = data.keys() 
        csv_writer.writerow(header) 
        count += 1
        
    # Writing data of CSV file 

    csv_writer.writerow(data.values()) 


with open('flipkart.csv') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    rows = list(reader)
i,j = 0,2
while i in range(len(rows)):
    try:
        name = rows[j][0]
        #name = " ".join(name.split(' ')[0:2])
        #print(name)
        #print("name = ",name)
        names.append(name)
        i += 1
        j += 2
    except:
            break


In [ ]:
print("Best results",len(names))
#print(names,len(names))
if len(names) <= 10:
    flipkart_url = "https://www.flipkart.com/search?q=" + q
    print(flipkart_url)
    uClient = uReq(flipkart_url)
    flipkartPage = uClient.read()
    uClient.close()
    flipkart_html = bs(flipkartPage, "html.parser")
    bigboxes = flipkart_html.find_all("a", {"class": "s1Q9rs"})
    soup = BeautifulSoup(flipkartPage, 'html.parser')
    info = soup.select("[class~=s1Q9rs]")
    flipPrices = soup.select("[class =_30jeq3]")
    prodNames = [i.get('title') for i in info]
    names = prodNames
    df = pd.DataFrame(list(zip(prodNames, flipPrices)), 
                columns =['product_name', 'Flipkart_price']) 
    df.to_csv('flipkart.csv')
    print(df)

else:
    with open('flipkart.csv') as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        rows = list(reader)

Best results 0
https://www.flipkart.com/search?q=laptop
Empty DataFrame
Columns: [product_name, Flipkart_price]
Index: []


In [ ]:
    i,j = 0,2
    while i in range(len(rows)):
        try:
            price = rows[j][4]
            #price = price[i].text
            #print("price = ",price)
            flipPrices.append(price)
            i += 1
            j += 2
        except:
            break
    df = pd.DataFrame(list(zip(names, flipPrices)), 
            columns =['Product_name', 'Flipkart_price'])

    print(df)
    df.to_csv('flipkart.csv')

data_file.close()

Empty DataFrame
Columns: [Product_name, Flipkart_price]
Index: []


In [ ]:
data = pd.read_csv("flipkart.csv", usecols = ['title','keySpecs','rating','ratingCount','price'])
data

,title,keySpecs,rating,ratingCount,price
0,ASUS VivoBook 15 (2022) Core i5 10th Gen - (8 ...,"['Intel Core i5 Processor (10th Gen)', '8 GB D...",4.3,4388,42990
1,ASUS TUF Gaming F15 Core i5 10th Gen - (8 GB/5...,"['Intel Core i5 Processor (10th Gen)', '8 GB D...",4.4,2813,55990
2,MSI Core i7 11th Gen - (8 GB/512 GB SSD/Window...,"['Intel Core i7 Processor (11th Gen)', '8 GB D...",4.4,717,77990
3,ASUS VivoBook 15 (2022) Core i3 10th Gen - (8 ...,"['Intel Core i3 Processor (10th Gen)', '8 GB D...",4.3,6479,35990
4,HP Athlon Dual Core 3050U - (8 GB/512 GB SSD/W...,"['AMD Athlon Dual Core Processor', '8 GB DDR4 ...",4.3,129,28990
5,acer Aspire 7 Ryzen 5 Hexa Core AMD R5-5500U -...,"['Free upgrade to Windows 11 when available', ...",4.5,4249,47990
6,realme Book(Slim) Intel Evo Core i5 11th Gen -...,['Powered by 11th Gen Intel Evo Core i5 Proces...,4.4,6104,59999
7,RedmiBook 15 Core i3 11th Gen - (8 GB/256 GB S...,"['Intel Core i3 Processor (11th Gen)', '8 GB D...",4.2,2427,27990
8,HP Core i3 11th Gen - (8 GB/512 GB SSD/Windows...,"['Intel Core i3 Processor (11th Gen)', '8 GB D...",4.3,2448,40999
9,MSI Bravo 15 Ryzen 5 Hexa Core AMD R5-5600H - ...,"['AMD Ryzen 5 Hexa Core Processor', '8 GB DDR4...",4.4,1334,54990


In [ ]:
import pandas as pd
df = pd.read_csv("test.csv", sep=",")
df.head(100)